In [ ]:
# support async in notebooks, read more about why we need this here: https://pypi.org/project/nest-asyncio/
import nest_asyncio

nest_asyncio.apply()

### Logging

In [ ]:
# configure logging
import logging

logging.basicConfig(
    format="{asctime} - {name} - {levelname} - {message}", style="{", level="INFO"
)

### Connection

The ingestion library requires a connection object from the Firebolt Python SDK. The following cells demonstrate how to create one from the SDK's ResourceManager. Please refer to the SDK documentaiton for additional ways to generate a connection object. If you already have a connection object, you can skip this section!

create a .env file in the same directory as this notebook with the following contents (fill in values):
```
FIREBOLT_USER=''
FIREBOLT_PASSWORD=''
FIREBOLT_SERVER=''
FIREBOLT_DEFAULT_REGION=''
```


In [ ]:
from firebolt.service.manager import ResourceManager
from firebolt.common import Settings

rm = ResourceManager()

In [ ]:
import time

# optionally replace "" with the database and engine to which you want to connect.
# otherwise, run this as is to create a temp database and engine for demo purposes.
default_name = f"temp_{int(time.time())}"
database_name = "" or default_name
engine_name = "" or default_name
table_name = "" or default_name

database = rm.databases.create(name=database_name, region="us-east-1")
engine = rm.engines.create(name=engine_name)
engine.attach_to_database(database=rm.databases.get_by_name(name=database_name))
engine = engine.start()
connection = engine.get_connection()

### Table configuration - Python


Define a table using native Python.

In [ ]:
from firebolt_ingest.model.table import Table, Column, Partition

table = Table(
    table_name=table_name,
    columns=[Column(name="col_1", type="STRING"), Column(name="col_2", type="INT")],
    primary_index=["col_1"],
    file_type="PARQUET",
    object_pattern=["*.parquet"],
)

### Table configuration - YAML

Define a table in YAML. The first cell uses the previous table object and converts it into YAML. The second cell shows how to take a YAML file and parse it into a `Table` object.

In [ ]:
import yaml

table_yaml_config = yaml.dump(table.dict())

print(table_yaml_config)

In [ ]:
table = Table.parse_yaml(table_yaml_config)
table

### Create internal table

In [ ]:
from firebolt_ingest.service.table import TableService

ts = TableService(connection)
ts.create_internal_table(table=table)